This notebook shows the functions provided by TINC to create parameter spaces and data pools from configuration and output files. There are two methods:

  * Using the output data
  * Using configuration files
  
Using the output can be very convenient as you can create a DataPool automatically to explore the data, but there are cases where you might need to extract the parameter space from configuration files instead, in cases where the parameters are not present in the output data.

In [39]:
from tinc import *
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [40]:
#Load  tinc data pools for the separate heating and cooling grids in T vs chempot space
heating_data_dir = '/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/heating'    
cooling_data_dir = '/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/mc_coarse_new_scel/cooling'
heating_dp, heating_ps  = create_datapool_from_output(heating_data_dir, "results.json", ignore_params=['Beta'])
cooling_dp, cooling_ps = create_datapool_from_output(cooling_data_dir, "results.json", ignore_params=['Beta'])

Found more than one potential parameter in files:['Beta', 'T'] 
Using:T 
Found more than one potential parameter in files:['Beta', 'T'] 
Using:T 


In [35]:
#List fields that can be queried:
heating_dp.list_fields()


['<atom_frac(N)>',
 '<atom_frac(Zr)>',
 '<comp(a)>',
 '<comp_n(N)>',
 '<comp_n(Va)>',
 '<comp_n(Zr)>',
 '<formation_energy>',
 '<potential_energy>',
 '<site_frac(N)>',
 '<site_frac(Va)>',
 '<site_frac(Zr)>',
 'Beta',
 'N_avg_samples',
 'N_equil_samples',
 'T',
 'heat_capacity',
 'is_converged',
 'is_equilibrated',
 'param_chem_pot(a)',
 'prec(<atom_frac(N)>)',
 'prec(<atom_frac(Zr)>)',
 'prec(<comp(a)>)',
 'prec(<comp_n(N)>)',
 'prec(<comp_n(Va)>)',
 'prec(<comp_n(Zr)>)',
 'prec(<formation_energy>)',
 'prec(<potential_energy>)',
 'prec(<site_frac(N)>)',
 'prec(<site_frac(Va)>)',
 'prec(<site_frac(Zr)>)',
 'susc_n(N,N)',
 'susc_n(N,Va)',
 'susc_n(N,Zr)',
 'susc_n(S,N)',
 'susc_n(S,Va)',
 'susc_n(S,Zr)',
 'susc_n(Va,Va)',
 'susc_n(Va,Zr)',
 'susc_n(Zr,Zr)',
 'susc_x(S,a)',
 'susc_x(a,a)',
 'integ_grand_free_energy']

In [36]:
heating_dp.get_slice("integ_grand_free_energy", "param_chem_pot(a)")
#dp.get_slice("<comp_n(N)>", "T")


masked_array(data=[ 3.7538629e-02,  3.7411947e-02,  3.7268307e-02,
                    3.7096158e-02,  3.6900580e-02,  3.6666315e-02,
                    3.6398679e-02,  3.6083251e-02,  3.5722330e-02,
                    3.5291769e-02,  3.4795910e-02,  3.4211397e-02,
                    3.3534102e-02,  3.2730497e-02,  3.1792656e-02,
                    3.0685818e-02,  2.9381581e-02,  2.7847877e-02,
                    2.6039608e-02,  2.3902917e-02,  2.1389842e-02,
                    1.8424008e-02,  1.4936008e-02,  1.0837190e-02,
                    6.0306857e-03,  4.2946354e-04, -6.0630576e-03,
                   -1.3496105e-02, -2.1884905e-02, -3.1212978e-02,
                   -4.1387841e-02, -5.2349430e-02, -6.4032316e-02,
                   -7.6353125e-02, -8.9274406e-02, -1.0272570e-01,
                   -1.1666850e-01, -1.3106889e-01, -1.4590499e-01,
                   -1.6113611e-01, -1.7674707e-01, -1.9272780e-01,
                   -2.0906168e-01, -2.2571935e-01, -2.4270289e

In [41]:
#Create meshgrid of chemical potential and temperature
heating_chempot_grid, heating_temperature_grid = np.meshgrid(heating_ps.get_dimension('param_chem_pot(a)').values, heating_ps.get_dimension("T").values)
print(heating_chempot_grid)
print(heating_temperature_grid)

#Initialize integrated free energy grid
heating_integrated_free_energy = np.zeros(heating_chempot_grid.shape)
print(heating_integrated_free_energy.shape)


#Same for cooling
cooling_chempot_grid, cooling_temperature_grid = np.meshgrid(cooling_ps.get_dimension("param_chem_pot(a)").values, cooling_ps.get_dimension("T").values)
cooling_integrated_free_energy = np.zeros(cooling_chempot_grid.shape)


[[-1.    -0.975 -0.95  ...  0.95   0.975  1.   ]
 [-1.    -0.975 -0.95  ...  0.95   0.975  1.   ]
 [-1.    -0.975 -0.95  ...  0.95   0.975  1.   ]
 ...
 [-1.    -0.975 -0.95  ...  0.95   0.975  1.   ]
 [-1.    -0.975 -0.95  ...  0.95   0.975  1.   ]
 [-1.    -0.975 -0.95  ...  0.95   0.975  1.   ]]
[[2000. 2000. 2000. ... 2000. 2000. 2000.]
 [1980. 1980. 1980. ... 1980. 1980. 1980.]
 [1960. 1960. 1960. ... 1960. 1960. 1960.]
 ...
 [  60.   60.   60. ...   60.   60.   60.]
 [  40.   40.   40. ...   40.   40.   40.]
 [  20.   20.   20. ...   20.   20.   20.]]
(100, 81)


In [48]:
[i.id for i in heating_ps.get_dimensions()]

['T', 'param_chem_pot(a)']

In [42]:
for value in heating_ps.get_dimension("param_chem_pot(a)").values:
    
    #Set the current value of the tinc parameter "param_chem_pot(a)" to "value" 
    heating_ps.get_parameter("param_chem_pot(a)").value = value
    
    #Slice integrated grand canonical free energy vs T for the fixed chemical potential
    tmp_heating_integ_free_energy = heating_dp.get_slice("integ_grand_free_energy","T")

    #Find the index of the current chemical potential in the vetor of chemical potential values (held by the tinc parameter "param_chem_pot(a)")
    chempot_index = heating_ps.get_dimension("param_chem_pot(a)").values.index(value)

    #fill in the integrated grand canonical free energy grid for fixed chemical potential
    heating_integrated_free_energy[:,chempot_index] = tmp_heating_integ_free_energy


#Same for the cooling free energy surface. 
for value in cooling_ps.get_dimension("param_chem_pot(a)").values:
    
    cooling_ps.get_parameter("param_chem_pot(a)").value = value
    
    tmp_cooling_integ_free_energy = cooling_dp.get_slice("integ_grand_free_energy","T")

    chempot_index = cooling_ps.get_dimension("param_chem_pot(a)").values.index(value)

    cooling_integrated_free_energy[:,chempot_index] = tmp_cooling_integ_free_energy



KeyError: 'integ_grand_free_energy'

In [ ]:
#plot the two free energy surfaces:
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,linewidth=0, antialiased=False)

def plot_dim(dim_name, fixed_dim, dp):
    ps = dp.get_parameter_space()
    xlabel(dim_name)

    for value in ps.get_dimension(fixed_dim).values:
        ps.get_parameter(fixed_dim).value = value
        plot(ps.get_parameter(dim_name).values, dp.get_slice("<formation_energy>", dim_name))
        plot(ps.get_parameter(dim_name).values, dp.get_slice("<formation_energy>", dim_name))

    legend(ps.get_dimension(fixed_dim).values)

    
    
figure(figsize=(12,4))
subplot(1,2, 1)
    
plot_dim("param_chem_pot(a)", "T", dp)
    
subplot(1,2, 2)
    
plot_dim("T", "param_chem_pot(a)", dp)



In [ ]:
heating_dp.list_fields()
dp.get_slice("<comp_n(N)>", "T")
dp.get_slice("<comp_n(N)>","param_chem_pot(a)")[40]
#ps.get_dimension("T").value



In [ ]:
ps.get_dimension("param_chem_pot(a)").values.index(0.0)

## tl;dr Create datapools from output

You can create a data pool directly from results using ```create_datapool_from_output()```. This function returns the datapool and parameter space extracted from the data files.

# Extracting from configuration files

To extract a parameter space from configuration files, you must provide a data root directory and the name of the configuration file that can be found in subdirectories. This assumes all configuration file names are the same.

You will also need to describe how to extract the information from the configuration files. This is done by specifying the keys where the parameter data is found. For example if the configuration files look like:

```json
{
  "driver" : {
    "mode" : "incremental", 
    "motif" : {
      "configname" : "restricted_auto",
      "_configname" : "SCEL1_1_1_1_0_0_0/0",
      "_configdof" : "$HOME/laptop_share/NbO_rocksalt_gs/mc_runs/fit_13.02/coarse_grid/set2_cooling_grid2/A_3.9B_-19.1/conditions.298/tto/final_state.json"
    },
    "initial_conditions" : {
      "param_chem_pot" : {
        "a" : 3.90,
        "b" : -19.80
      },
      "temperature" : 20.0,
      "tolerance" : 0.001
    },
    "final_conditions" : {
      "param_chem_pot" : {
        "a" : 3.90,
        "b" : -19.80
      },
      "temperature" :2800.0,
      "tolerance" : 0.001
    },
    "incremental_conditions" : {
      "param_chem_pot" : {
        "a" : 0.0,
        "b" : 0.0
      },
      "temperature" : 10.0,
      "tolerance" : 0.001
    }
  }
}

```

You specify the starting value key as: ```driver/initial_conditions/*``` because the starting values are a list within the "driver" and " initial_conditions" keys. A similar string needs to be constructed for end and increment keys.


Current limitations:
  * JSON only
  * space must be described by its boundaries and the incremement
  * Limited format to describe how to extract the information. Currently values must be leaf nodes.

TODO... Ignore below...